# Exercise: Eco-Friendly Product Launch
From: https://estudijas.rtu.lv/mod/assign/view.php?id=4116908

## Background:

Imagine you're part of a European eco-friendly startup, Green Innovate EU, focusing on launching sustainable products. Your team is ready to introduce two new items: a biodegradable water bottle BIO_BOT and a solar-powered phone charger SOL_CHA. Both products align with the company's mission to reduce plastic waste and promote renewable energy sources. The challenge lies in maximizing profit while adhering to budget constraints and production capabilities.

Problem Statement:

Green Innovate EU has a budget of €50,000 for the first production run. The cost to produce one biodegradable water bottle is €3, and the cost for one solar-powered phone charger is €8. The expected profit from each water bottle is €5, and from each charger is €12. The market research team has indicated that due to the current market trends and consumer demand, the company should produce at least twice as many water bottles as chargers. Moreover, the production facility's constraints allow for the manufacture of a maximum of 10,000 water bottles and 4,000 chargers for this run.

Objectives:

Determine how many of each product the startup should produce to maximize profit.
Ensure that the production plan respects the budget limit, production capacity, and market demand insights.
Requirements:

Let
X1
 represent the number of biodegradable water bottles to produce.
Let
X2
 represent the number of solar-powered phone chargers to produce.
Maximize the total profit
P
=
5*X1+12*X2

Subject to:
Budget constraint:
3X1+8X2≤50_000

Production constraints:
X1≤10_000
,
X2≤4_000

Market demand constraint:
X1≥2X2

Task:

Use linear programming techniques to find the optimal production quantities of water bottles and chargers. Illustrate your solution process and discuss the implications of your results for the startup's strategy.



Allowed approaches:

While it would be extremely impressive, if you could implement a simplex algorithm from scratch, you are allowed to use libraries for this solution.
Also you are allowed to try different approach, even some sort of brute-force solution.
Also you are allowed to use pen and paper to solve this problem by hand and then verify solution.
An interesting attempt would be to use some sort of hill-climbing approach.

## Pulp Library

[PuLP](https://coin-or.github.io/pulp/) is an open-source linear programming package for Python. It allows users to describe optimization problems in a Python-based modeling language.

PyPi for PuLP: https://pypi.org/project/PuLP/ - last update September 18th,2025

In [1]:
# check if we have pulp if not we install it
try:
    import pulp
except ImportError:
    print("pulp not installed")
    # import pip
    # pip.main(['install', 'pulp'])
    # import pulp

pulp not installed


In [2]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 98.8 MB/s eta 0:00:00


In [4]:
# from pulp import * # this is bad practice but we do it for the sake of the tutorial
from pulp import LpProblem, LpMaximize, LpVariable, LpStatus, value   # this is better practice
# using * means we pollute the namespace with all the pulp functions
# this could cause problems if we do this with multiple libraries - huge namespace conflicts

# Create the problem variable
prob = LpProblem("Green Innovate EU", LpMaximize)

# Define the decision variables
x1 = LpVariable("BIO_BOT", lowBound=0, cat='Float') # so Float leads to easier to solve problems than Integer
x2 = LpVariable("SOL_CHA", lowBound=0, cat='Float')

# Why Integer requirememt would make it harder? It would be because our solution space could lie inside our boundary lines (drawn by our constraint linear equations)

# Define the objective function
prob += 5*x1 + 12*x2

# Define the constraints
prob += 3*x1 + 8*x2 <= 50000
prob += x1 <= 10000
prob += x2 <= 4000
prob += x1 >= 2*x2

# Solve the problem
prob.solve()

# Print the optimal solution
print("Optimal production quantities:")
print("BIO_BOT:", value(x1))
print("SOL_CHA:", value(x2))
print("Total profit:", value(prob.objective))

Optimal production quantities:
BIO_BOT: 10000.0
SOL_CHA: 2500.0
Total profit: 80000.0


## Brute force approach

In [5]:
# let's try to find max between 0 and 10000 for x1 and 0 and 4000 for x2
# of course this could solve it for integeral values, not so good for float values where our search space is unlimited :)
max_profit = 0
for x1 in range(0, 10001):
    for x2 in range(0, 4001):
        if 3*x1 + 8*x2 <= 50000 and x1 >= 2*x2:
            profit = 5*x1 + 12*x2
            if profit > max_profit:
                max_profit = profit
                max_x1 = x1
                max_x2 = x2
print("Optimal production quantities:")
print("BIO_BOT:", max_x1)
print("SOL_CHA:", max_x2)
print("Total profit:", max_profit)

Optimal production quantities:
BIO_BOT: 10000
SOL_CHA: 2500
Total profit: 80000


## Hill-Climbing approach



In [12]:
# we can start moving quicker by making bigger steps in the search space initially
# because we have linear constraints we do not expect to see any valleys or peaks
max_profit = 0
total_solutions_explored = 0
legal_solutions_explored = 0
STEP = 10 # so changing the step to be steeper could lose us our solution
for x1 in range(0, 10001, STEP):
    for x2 in range(0, 4001, STEP):
        total_solutions_explored += 1
        if 3*x1 + 8*x2 <= 50000 and x1 >= 2*x2:
            legal_solutions_explored += 1
            profit = 5*x1 + 12*x2
            if profit > max_profit:
                max_profit = profit
                max_x1 = x1
                max_x2 = x2
                # we could add a condition for early exit when we reach satisfactory profit

print("Optimal production quantities:")
print("BIO_BOT:", max_x1)
print("SOL_CHA:", max_x2)
print("Total profit:", max_profit)
print("Total solutions explored:", total_solutions_explored)
print("Legal solutions explored:", legal_solutions_explored)

Optimal production quantities:
BIO_BOT: 10000
SOL_CHA: 2500
Total profit: 80000
Total solutions explored: 401401
Legal solutions explored: 215108


In [16]:
# That worked but it is possible that we missed the optimal solution if it was in the middle of the step
# idea is to make the step adjustable as we get closer to the solution
# our minimal steps are 1, 1
# we can start with 100, 100
# if we are close to the solution we can reduce the step to 1, 1
# we can use increase in profit as a measure of how close we are to the solution
max_profit = 0
step = 10
profit_increase = 0
total_solutions_explored = 0
legal_solutions_explored = 0

# lets caclucate potential profit if we relax all constraints except keep number of units

x1_cand = 10_000
x2_cand = 4_000
potential_profit = 5*x1_cand + 12*x2_cand
steps = []
for x1 in range(0, 10001, step):
    for x2 in range(0, 4001, step):
        total_solutions_explored += 1
        if 3*x1 + 8*x2 <= 50000 and x1 >= 2*x2:
            legal_solutions_explored += 1
            profit = 5*x1 + 12*x2
            if profit > max_profit:
                # profit_increase = profit - max_profit
                max_profit = profit
                max_x1 = x1
                max_x2 = x2
                # now we could adjust step based on profit_increase
                # ideas as profit increase get smaller we can reduce the step
                # now we could adjust step based on how close our max_profit is to potential_profit
                # if we are close we can reduce the step
                # if we are far we can increase the step
                # TODO our new_step should be based on how close we are to the potential_profit
                # currently it is very ad hoc and not based on any theory...
                # notably 10_000 is a magic number at the moment
                new_step = max(min(100, int((potential_profit - max_profit)/10_000)), 1)
                if new_step != step:
                    step = new_step
                    steps.append(step)
print("Optimal production quantities:")
print("BIO_BOT:", max_x1)
print("SOL_CHA:", max_x2)
print("Total profit:", max_profit)
print("Total solutions explored:", total_solutions_explored)
print("Legal solutions explored:", legal_solutions_explored)
print("Steps taken", steps)

Optimal production quantities:
BIO_BOT: 10000
SOL_CHA: 2500
Total profit: 80000
Total solutions explored: 1818812
Legal solutions explored: 1256421
Steps taken [9, 8, 7, 6, 5, 4, 3, 2, 1]


In [14]:
len(steps)

9

In [15]:
steps

[9, 8, 7, 6, 5, 4, 3, 2, 1]

## Ideas - Branch and bound approach

We relax the requirement that the number of water bottles must be an integer. We can then solve the problem using the simplex algorithm. We then round the solution to the nearest integer and check if the solution is feasible. If it is, we check if the solution is better than the best solution found so far. If it is, we update the best solution. We then branch on the variable that is not an integer and repeat the process. We stop when we have checked all branches or when we can prove that the best solution found so far is optimal.


## Plotting Solutions

In [17]:
# let's use Plotly to visualize the constraints
import plotly.graph_objects as go
import numpy as np

x1 = np.linspace(0, 10000, 100)
x2 = np.linspace(0, 4000, 100)
X1, X2 = np.meshgrid(x1, x2)
Z = 5*X1 + 12*X2

fig = go.Figure(data=[go.Surface(z=Z, x=X1, y=X2)])
fig.update_layout(scene=dict(xaxis_title='BIO_BOT', yaxis_title='SOL_CHA', zaxis_title='Profit'))
fig.show()
# 3d plot is kind of hard to read and understand
# let's use contour plot instead

In [18]:
# instead of 3d graph lets use color map to represent the profit
fig = go.Figure(data=go.Contour(z=Z, x=x1, y=x2))
fig.update_layout(xaxis_title='BIO_BOT', yaxis_title='SOL_CHA', title='Profit')
fig.show()

In [23]:
# let's add two other constraints to the graph
# if 3*x1 + 8*x2 <= 50000
# converting to x2 = (50000 - 3*x1)/8
# and x1 >= 2*x2
# converting to x2 = x1/2
# x1 is BIO_BOT
# x2 is SOL_CHA
# let's draw 2-d plot for each constraint
# let's plot it together with the profit plot as contour plot
x1 = np.linspace(0, 10000, 100)
x2 = np.linspace(0, 4000, 100)
X1, X2 = np.meshgrid(x1, x2)
Z = 5*X1 + 12*X2
Z1 = (50000 - 3*X1)/8
Z2 = X1/2

# since we know the solution is 10_000, 4_000 and we know the profit is 80_000
# we can use this information to plot possible solution space
SOLUTION = (80000 - 5*X1)/12


# now let's plot the profit and the constraints
fig = go.Figure(data=go.Contour(z=Z, x=x1, y=x2, colorscale='Viridis', colorbar=dict(title='Profit')))
# fig.add_trace(go.Contour(z=Z1, x=x1, y=x2, colorscale='Blues', colorbar=dict(title='Constraint 1')))
# fig.add_trace(go.Contour(z=Z2, x=x1, y=x2, colorscale='Reds', colorbar=dict(title='Constraint 2')))
# lets add Z1 and Z2 as separate Scatter plots
fig.add_trace(go.Scatter(x=x1, y=Z1[0], mode='lines', name='Constraint 1', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=x1, y=Z2[0], mode='lines', name='Constraint 2', line=dict(color='red')))

# let's plot SOLUTION
fig.add_trace(go.Scatter(x=x1, y=SOLUTION[0], mode='lines', name='SOLUTION', line=dict(color='green')))
fig.update_layout(xaxis_title='BIO_BOT', yaxis_title='SOL_CHA', title='Profit and Constraints')


# adjust height of the plot to 1000
fig.update_layout(height=1000)
# move legend up
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.0))
fig.show()



In [24]:
# so 6 variables with 10_000 possible values would require brute force to use
10_000**6

1000000000000000000000000